# Question 1

In [59]:
import gurobipy as gp
from gurobipy import GRB

In [60]:
machineProduct, cost, time, setup = gp.multidict(
{
    (1, 1): [13, 0.4, 55],
    (1, 2): [9, 1.1, 93],
    (1, 3): [10, 0.9, 60],
    (2, 1): [11, 0.5, 65],
    (2, 2): [12, 1.2, 58],
    (2, 3): [8, 1.3, 75]
})


demand = {
    1: 3,  
    2: 7,  
    3: 4   
}

machTime = {
    1: 8,
    2: 6
}

In [61]:
mprodMachine = gp.Model('prodMachine')

numberProd = mprodMachine.addVars(machineProduct, vtype='I', name='numberProd') 
assign = mprodMachine.addVars(machineProduct, vtype='B', name='assign') 

# Objective function
TotCost = gp.quicksum(cost[i,j]*numberProd[i,j] + setup[i,j]*assign[i,j] for i,j in machineProduct)

mprodMachine.setObjective(TotCost, GRB.MINIMIZE)

# Constraint
for j in range(1, 4):
    mprodMachine.addConstr(
        sum(numberProd[i, j] for i,j in machineProduct.select('*', j)) == demand[j],
        name="demand"
        )

for i in range(1, 3):
    mprodMachine.addConstr(
        sum(time[i,j] * numberProd[i, j] for i,j in machineProduct.select(i, '*')) \
            <= machTime[i],
        name="time"
        )

for i in range(1, 3):
    for j in range(1,4):
        mprodMachine.addConstr(
            numberProd[i, j] <= demand[j] * assign[i, j],
            name="Link"
            )
        
# Verify model formulation
mprodMachine.write('prodMachine.lp')

mprodMachine.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 9 5900HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 11 rows, 12 columns and 24 nonzeros
Model fingerprint: 0x526d63f2
Variable types: 0 continuous, 12 integer (6 binary)
Coefficient statistics:
  Matrix range     [4e-01, 7e+00]
  Objective range  [8e+00, 9e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 8e+00]
Found heuristic solution: objective 546.0000000
Presolve removed 6 rows and 6 columns
Presolve time: 0.00s
Presolved: 5 rows, 6 columns, 12 nonzeros
Variable types: 0 continuous, 6 integer (3 binary)

Root relaxation: objective 4.206000e+02, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  4

We want to print the variables with a value of greater than 0 because this is the tuple that gurobi's optimization says we should take

In [62]:
for x in mprodMachine.getVars():
    if (x.x != 0):
        print(x.varName, x.x)
print('Total cost: ' + str(mprodMachine.objVal))

numberProd[1,2] 4.0
numberProd[1,3] 4.0
numberProd[2,1] 3.0
numberProd[2,2] 3.0
assign[1,2] 1.0
assign[1,3] 1.0
assign[2,1] 1.0
assign[2,2] 1.0
Total cost: 421.0


# Question 2

In [63]:
supplyResource, production = gp.multidict(
{
    (1, 1): 2.00,
    (1, 2): 2.80,
    (1, 3): 1.70,
    (1, 4): 2.40,
    (2, 1): 1.80,
    (2, 2): 2.30,
    (2, 3): 1.75,
    (2, 4): 1.90,
    (3, 1): 2.30,
    (3, 2): 2.20,
    (3, 3): 1.60,
    (3, 4): 2.60,
    (4, 1): 2.10,
    (4, 2): 2.60,
    (4, 3): 1.90,
    (4, 4): 2.40,
})


supply, cost, truck, avail = gp.multidict(
{
    1: [22, 1500, 1500],
    2: [21, 1700, 2000],
    3: [22, 1500, 1500],
    4: [23, 1400, 1800]
})

demand = {
    1: 750,
    2: 800,
    3: 1000,
    4: 300
}

In [64]:
mgas = gp.Model('gas')

numberProd = mgas.addVars(supplyResource, vtype='I', name='numberProd') 
assign = mgas.addVars(demand, vtype='B', name='assign') 

# Objective function
TotCost = gp.quicksum(cost[i]*numberProd[i,j] for i,j in supplyResource)+\
      gp.quicksum(truck[i]*assign[i] for i in supply)

mgas.setObjective(TotCost, GRB.MINIMIZE)

# Constraint
for j in range(1, 5):
    mgas.addConstr(
        sum(production[i, j] * numberProd[i, j] for i,j in supplyResource.select('*', j))\
              >= demand[j],
        name="Demand"
        )

for i in range(1, 5):
    mgas.addConstr(
        sum(numberProd[i, j] for i,j in supplyResource.select(i, '*'))\
            <= avail[i] * assign[i],
        name="Availiability"
        )

for i in range(1, 5):
    mgas.addConstr(
        sum(numberProd[i, j] for i,j in supplyResource.select(i, '*'))\
            >= 500 * assign[i],
        name="supply"
        )
        
# Verify model formulation
mgas.write('gas.lp')

mgas.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 9 5900HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 12 rows, 20 columns and 56 nonzeros
Model fingerprint: 0x4567b16b
Variable types: 0 continuous, 20 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [2e+01, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+02, 1e+03]
Found heuristic solution: objective 35758.000000
Presolve time: 0.00s
Presolved: 12 rows, 20 columns, 56 nonzeros
Variable types: 0 continuous, 20 integer (4 binary)

Root relaxation: objective 2.921099e+04, 8 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 29210.9890    0    7 35758.0000 2921

In [65]:
for x in mgas.getVars():
    if (x.x != 0):
        print(x.varName, x.x)
print('Total cost: ' + str(mgas.objVal))

numberProd[4,1] 358.0
numberProd[4,2] 308.0
numberProd[4,3] 527.0
numberProd[4,4] 125.0
assign[4] 1.0
Total cost: 31714.0


In [66]:
358.0+308.0+527.0+125.0

1318.0

Since only need location 4, which is AL, we sum up and get 1318 barrels

# Question 3

In [125]:
week, demand = gp.multidict(
{
    1: 400,
    2: 150,
    3: 200,
    4: 350
})

In [128]:
mMega_Bucks = gp.Model('Mega-Bucks')

beginInv = mMega_Bucks.addVars(week, vtype='I', name='BeginInventory') 
endInv = mMega_Bucks.addVars(week, vtype='I', name='EndInventory') 
numInv = mMega_Bucks.addVars(week, vtype='I', name='numberOrder') 
assign = mMega_Bucks.addVars(week, vtype='B', name='assign') 

# Objective function
TotCost = gp.quicksum(125*numInv[i]for i in week)+\
    gp.quicksum(50*assign[i] for i in week)+\
    gp.quicksum(15*endInv[i] for i in week)/100

mMega_Bucks.setObjective(TotCost, GRB.MINIMIZE)

# Constraint
for i in range(2, 5):
    mMega_Bucks.addConstr(
        beginInv[i] == endInv[i-1],
        name="Begin"
        )
        
for i in range(1, 5):
    mMega_Bucks.addConstr(
        endInv[i] == beginInv[i] + 100*numInv[i]-demand[i],
        name="End"
        )

M = 1000  
for i in range(1, 5):
    mMega_Bucks.addConstr(
        numInv[i] <= M*assign[i],
        name="Batch"
        )

mMega_Bucks.addConstr(beginInv[1]==0)     

# Verify model formulation
mMega_Bucks.write('Mega_Bucks.lp')

mMega_Bucks.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 9 5900HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 12 rows, 16 columns and 27 nonzeros
Model fingerprint: 0x5916448b
Variable types: 0 continuous, 16 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e-01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+02, 4e+02]
Found heuristic solution: objective 1730.0000000
Presolve removed 7 rows and 8 columns
Presolve time: 0.00s
Presolved: 5 rows, 8 columns, 12 nonzeros
Found heuristic solution: objective 1590.0000000
Variable types: 0 continuous, 8 integer (3 binary)

Root relaxation: objective 1.510100e+03, 6 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbe

In [129]:
for x in mMega_Bucks.getVars():
    if (x.x != 0):
        print(x.varName, x.x)
print('Total cost: ' + str(mMega_Bucks.objVal))

BeginInventory[2] 200.0
BeginInventory[3] 50.0
BeginInventory[4] 350.0
EndInventory[1] 200.0
EndInventory[2] 50.0
EndInventory[3] 350.0
numberOrder[1] 6.0
numberOrder[3] 5.0
assign[1] 1.0
assign[3] 1.0
Total cost: 1565.0


we need to order 600 pound in week 1 and 600 pound in week3